In [1]:
import os

base_script = '''#!/bin/bash

#SBATCH --job-name=c{part}
#SBATCH --nodes=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=8GB
#SBATCH --time=168:00:00
#SBATCH --mail-user=chris.ick@nyu.edu
#SBATCH --export=NONE
#SBATCH --output="outputs/o_{part}-%j.out"

module purge
module load anaconda3/2020.07

source activate rap_env
cd /home/ci411/RapAlignment
python process_beats.py --audio-path='/scratch/work/sonyc/marl/private_datasets/FMA/fma_large/fma_large/{part}' --jams-dir='/scratch/ci411/rap_data/jams_fma_large'
'''

In [2]:
base_script.format(part='000')

'#!/bin/bash\n\n#SBATCH --job-name=c000\n#SBATCH --nodes=1\n#SBATCH --cpus-per-task=1\n#SBATCH --mem=8GB\n#SBATCH --time=168:00:00\n#SBATCH --mail-user=chris.ick@nyu.edu\n#SBATCH --export=NONE\n#SBATCH --output="outputs/o_000-%j.out"\n\nmodule purge\nmodule load anaconda3/2020.07\n\nsource activate rap_env\ncd /home/ci411/RapAlignment\npython process_beats.py --audio-path=\'/scratch/work/sonyc/marl/private_datasets/FMA/fma_large/fma_large/000\' --jams-dir=\'/scratch/ci411/rap_data/jams_fma_large\'\n'

In [3]:
with open('scripts/runall.bat', 'w') as f:
    pass
for i in range(156):
    num = str(i).zfill(3)
    script = base_script.format(part=num)
    filename = 'part{}.sbatch'.format(num)
    with open(os.path.join('scripts',filename), 'w') as f:
        f.write(script)
    with open('scripts/runall.bat', 'a') as f:
        f.write('sbatch {}\n'.format(filename))


In [4]:
base_script.format(part=num)

'#!/bin/bash\n\n#SBATCH --job-name=c155\n#SBATCH --nodes=1\n#SBATCH --cpus-per-task=1\n#SBATCH --mem=8GB\n#SBATCH --time=168:00:00\n#SBATCH --mail-user=chris.ick@nyu.edu\n#SBATCH --export=NONE\n#SBATCH --output="outputs/o_155-%j.out"\n\nmodule purge\nmodule load anaconda3/2020.07\n\nsource activate rap_env\ncd /home/ci411/RapAlignment\npython process_beats.py --audio-path=\'/scratch/work/sonyc/marl/private_datasets/FMA/fma_large/fma_large/155\' --jams-dir=\'/scratch/ci411/rap_data/jams_fma_large\'\n'

In [20]:
redo_dirs = []

In [21]:
for path, subdirs, files in os.walk('scripts/outputs'):
    for file in files:
        if ('checkpoint' not in file and '.out' in file):
            filepath = os.path.join('scripts/outputs', file)
            with open(filepath, 'r') as f:
                if 'Format mp3 detected only with low score of 1, misdetection possible!' in f.read():
                    print(filepath)
                    redo_dirs.append(file.split('_')[1].split('-')[0])
                    file_root = filepath.split('.')[0]
                    os.rename(filepath, file_root+'.redo')
        
print(redo_dirs)

[]


In [18]:
with open('scripts/redo.bat', 'w') as f:
    pass
for num in redo_dirs:
    filename = 'part{}.sbatch'.format(num)
    with open('scripts/redo.bat', 'a') as f:
        f.write('sbatch {}\n'.format(filename))


    